In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hackathon-online-nlu-slot-filling/train.json
/kaggle/input/hackathon-online-nlu-slot-filling/small_test.csv
/kaggle/input/hackathon-online-nlu-slot-filling/test.csv
/kaggle/input/hackathon-online-nlu-slot-filling/valid.json
/kaggle/input/hackathon-online-nlu-slot-filling/slot_sample_submission.csv
/kaggle/input/valid-seccond/bert_stn (1).csv
/kaggle/input/validation-from-first/bert_stn.csv


In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 67.2 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 76.7 MB/s eta 0:00:00:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=87106d5c8d300997e0ea246539d4abd9bcc4ddbf0b48348008441f3086292bb5
  Stored in directory: /root/.cache/pip/wheels/b2/a1/b7/0d3b008d0c77cd57332d724b92cf7650b4185b493dc785f00a
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=d22568c132f5439e9179df94179844c1a5f6f59bf1c7ce9edde37279cf281b49
  Stored in directory: /root/.cache/pip/wheels/3d/7c/f1/f87f51d07a2513aa0fb1

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

In [ ]:
valid=pd.read_json("/kaggle/input/hackathon-online-nlu-slot-filling/valid.json")
train=pd.read_json("/kaggle/input/hackathon-online-nlu-slot-filling/train.json")
test=pd.read_csv("/kaggle/input/hackathon-online-nlu-slot-filling/test.csv")
sample=pd.read_csv("/kaggle/input/hackathon-online-nlu-slot-filling/slot_sample_submission.csv")


In [ ]:
train.head()

,utterance,slots,intent
0,will it be rainy in tenino,O O O B-condition_description O B-city,GetWeather
1,add steve albini album to my psychedelic rock ...,O B-artist I-artist B-music_item O B-playlist_...,AddToPlaylist
2,book a table for 3 at one of the restaurants i...,O O O O B-party_size_number O O O O B-restaura...,BookRestaurant
3,what s the weather forecast for belknap,O O O O O O B-city,GetWeather
4,play fereydoun farrokhzad best track,O B-artist I-artist B-sort B-music_item,PlayMusic


In [ ]:
valid.head()

,utterance,slots,intent
0,i d like to see the show onion sportsdome,O O O O O O B-object_type B-object_name I-obje...,SearchCreativeWork
1,give the current book 4 stars,O O B-object_select B-object_type B-rating_val...,RateBook
2,reserve a table for 8 in neighboring hitchland,O O O O B-party_size_number O B-spatial_relati...,BookRestaurant
3,show creative video game the boat is full,O O B-object_type I-object_type B-object_name ...,SearchCreativeWork
4,put once bitten twice bitten into the pulse of...,O B-entity_name I-entity_name I-entity_name I-...,AddToPlaylist


In [ ]:
test

,id,utterance
0,1,add this tune to my playlist guest list mashable
1,2,i d like to listen to the song the natural farmer
2,3,i need a table for 8 people at a restaurant in...
3,4,open the canciones del recuerdo playlist and p...
4,5,i want to hear something from the top-fifty by...
...,...,...
295,296,show weather forecast in west wildwood south c...
296,297,add this track to the playlist hits of the 70s
297,298,i want to eat at a food truck this week with m...
298,299,rate company for gertrude 4 of 6


In [ ]:
sample

,id_word,slot
0,1_1,O
1,1_2,O
2,1_3,B-music_item
3,1_4,NaN
4,1_5,NaN
...,...,...
2741,300_11,NaN
2742,300_12,NaN
2743,300_13,NaN
2744,300_14,NaN


In [ ]:
train

,utterance,slots,intent
0,will it be rainy in tenino,O O O B-condition_description O B-city,GetWeather
1,add steve albini album to my psychedelic rock ...,O B-artist I-artist B-music_item O B-playlist_...,AddToPlaylist
2,book a table for 3 at one of the restaurants i...,O O O O B-party_size_number O O O O B-restaura...,BookRestaurant
3,what s the weather forecast for belknap,O O O O O O B-city,GetWeather
4,play fereydoun farrokhzad best track,O B-artist I-artist B-sort B-music_item,PlayMusic
...,...,...,...
13079,show me the lights out saga,O O O B-object_name I-object_name B-object_type,SearchCreativeWork
13080,what is a weather of martinique a month from now,O O O O O B-country B-timeRange I-timeRange I-...,GetWeather
13081,i d like to reserve a spot at a bakery in guya...,O O O O O O O O O B-restaurant_type O B-countr...,BookRestaurant
13082,please find me the work serious awesomeness,O O O O O B-object_name I-object_name,SearchCreativeWork


#preprocess create dict

In [ ]:
import nltk
sentence="""At eight o'clock on Thursday morning
... Arthur didn't feel very good."""
tokens = nltk.word_tokenize(sentence)
tokens

['At',
 'eight',
 "o'clock",
 'on',
 'Thursday',
 'morning',
 '...',
 'Arthur',
 'did',
 "n't",
 'feel',
 'very',
 'good',
 '.']

In [ ]:
a=train["utterance"].iloc[0]
print(a)
tokens = nltk.word_tokenize(a)
tokens

will it be rainy in tenino


['will', 'it', 'be', 'rainy', 'in', 'tenino']

In [ ]:
b=train["slots"].iloc[0]
print(b)
tokens = nltk.word_tokenize(b)
tokens

O O O B-condition_description O B-city


['O', 'O', 'O', 'B-condition_description', 'O', 'B-city']

In [ ]:
all_sentence_id=[]
all_words=[]
all_slots=[]
def collect_word_n_slot(a_train_df,count):
    for idx,rows in a_train_df.iterrows():
        a_utterance=rows["utterance"]
        a_slots=rows["slots"]
        a_t_tokens=nltk.word_tokenize(a_utterance)
        a_t_slots=nltk.word_tokenize(a_slots)
        if len(a_t_tokens)==len(a_t_slots):
            for word in a_t_tokens:
                all_sentence_id.append(count)
                all_words.append(word)
            count+=1
            for slot in a_t_slots:
                all_slots.append(slot)

In [ ]:
collect_word_n_slot(train,0)
print(len(all_words),len(all_slots),len(all_sentence_id))
train_f_s_f=pd.DataFrame({"sentence_id":all_sentence_id,"words":all_words,"labels":all_slots})
train_f_s_f

113763 113763 113763


,sentence_id,words,labels
0,0,will,O
1,0,it,O
2,0,be,O
3,0,rainy,B-condition_description
4,0,in,O
...,...,...,...
113758,12722,elstak,I-artist
113759,12722,in,O
113760,12722,1988,B-year
113761,12722,on,O


In [ ]:
len(all_words),len(all_slots),len(all_sentence_id)

(113763, 113763, 113763)

In [ ]:
train_f_s_f=pd.DataFrame({"sentence_id":all_sentence_id,"words":all_words,"labels":all_slots})
train_f_s_f

,sentence_id,words,labels
0,0,will,O
1,0,it,O
2,0,be,O
3,0,rainy,B-condition_description
4,0,in,O
...,...,...,...
113758,12722,elstak,I-artist
113759,12722,in,O
113760,12722,1988,B-year
113761,12722,on,O


In [ ]:
train_f_s_f[train_f_s_f["sentence_id"]==2]

,sentence_id,words,labels
15,2,book,O
16,2,a,O
17,2,table,O
18,2,for,O
19,2,3,B-party_size_number
20,2,at,O
21,2,one,O
22,2,of,O
23,2,the,O
24,2,restaurants,B-restaurant_name


In [ ]:
from simpletransformers.ner import NERModel,NERArgs

custom_labels = ['O', 'B-album', 'I-album', 'B-artist', 'I-artist', 'B-best_rating', 'I-best_rating',
'B-city', 'I-city', 'B-condition_description', 'I-condition_description', 'B-condition_temperature',
'I-condition_temperature', 'B-country', 'I-country', 'B-cuisine', 'I-cuisine', 'B-current_location',
'I-current_location', 'B-entity_name', 'I-entity_name', 'B-facility', 'I-facility', 'B-genre', 'I-genre',
'B-geographic_poi', 'I-geographic_poi', 'B-location_name', 'I-location_name', 'B-movie_name',
'I-movie_name', 'B-movie_type', 'I-movie_type', 'B-music_item', 'I-music_item',
'B-object_location_type', 'I-object_location_type', 'B-object_name', 'I-object_name',
'B-object_part_of_series_type', 'I-object_part_of_series_type', 'B-object_select', 'I-object_select',
'B-object_type', 'I-object_type', 'B-party_size_description', 'I-party_size_description',
'B-party_size_number', 'I-party_size_number', 'B-playlist', 'I-playlist', 'B-playlist_owner',
'I-playlist_owner', 'B-poi', 'I-poi', 'B-rating_unit', 'I-rating_unit', 'B-rating_value', 'I-rating_value',
'B-restaurant_name', 'I-restaurant_name', 'B-restaurant_type', 'I-restaurant_type', 'B-served_dish',
'I-served_dish', 'B-service', 'I-service', 'B-sort', 'I-sort', 'B-spatial_relation',
'I-spatial_relation', 'B-state', 'I-state', 'B-timeRange', 'I-timeRange', 'B-track', 'I-track', 'B-year', 'I-year']
model_args = {
    "reprocess_input_data": True,
    'train_batch_size':11,
    "use_early_stopping": True,
    "early_stopping_delta": 0.01,
    "early_stopping_metric": "mcc",
    "early_stopping_metric_minimize": False,
    "early_stopping_patience": 2,
    "evaluate_during_training_steps": 500,
    "fp16": False,
    "overwrite_output_dir":True,
    'use_cached_eval_features' : False, 
    'max_seq_length': 128,
    'no_cache': True,
    "warmup_ratio":0.06,
    "num_train_epochs":3,
    "adam_epsilon":1e-08,
}

In [ ]:
model = NERModel(
    "bert", "dslim/bert-base-NER",args=model_args,labels=custom_labels,ignore_mismatched_sizes=True
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NER and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([79, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([79]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.train_model(train_f_s_f, show_running_loss=True, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/1157 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/1157 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/1157 [00:00<?, ?it/s]

(3471, 0.23693977420374449)

#0.20845 0.22 0.213  0.2369

In [ ]:
predictions, raw_outputs = model.predict(list(valid["utterance"]))

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
len(predictions)

200

In [ ]:
test.head()

,id,utterance
0,1,add this tune to my playlist guest list mashable
1,2,i d like to listen to the song the natural farmer
2,3,i need a table for 8 people at a restaurant in...
3,4,open the canciones del recuerdo playlist and p...
4,5,i want to hear something from the top-fifty by...


In [ ]:
preds,raw=model.predict(list(test["utterance"]))

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/38 [00:00<?, ?it/s]

In [ ]:
preds[0]

[{'add': 'O'},
 {'this': 'O'},
 {'tune': 'B-music_item'},
 {'to': 'O'},
 {'my': 'B-playlist_owner'},
 {'playlist': 'O'},
 {'guest': 'B-playlist'},
 {'list': 'I-playlist'},
 {'mashable': 'I-playlist'}]

In [ ]:
sample

,id_word,slot
0,1_1,O
1,1_2,O
2,1_3,B-music_item
3,1_4,NaN
4,1_5,NaN
...,...,...
2741,300_11,NaN
2742,300_12,NaN
2743,300_13,NaN
2744,300_14,NaN


In [ ]:
l={'add': 'O'}
l.popitem()[1]

'O'

In [ ]:
def get_ans(preds):
    res=[]
    for i in preds:
        for j in i:
            res.append(j.popitem()[1])
    return res

In [ ]:
res=get_ans(preds)

In [ ]:
sample["slot"]=res

In [ ]:
sample

,id_word,slot
0,1_1,O
1,1_2,O
2,1_3,B-music_item
3,1_4,O
4,1_5,B-playlist_owner
...,...,...
2741,300_11,B-city
2742,300_12,I-city
2743,300_13,O
2744,300_14,B-country


In [ ]:
valid_9810=pd.read_csv("/kaggle/input/valid-seccond/bert_stn (1).csv")
valid_0977=pd.read_csv("/kaggle/input/validation-from-first/bert_stn.csv")
valid_0977

,id_word,slot
0,1_1,O
1,1_2,O
2,1_3,B-music_item
3,1_4,O
4,1_5,B-playlist_owner
...,...,...
2741,300_11,B-country
2742,300_12,I-country
2743,300_13,I-country
2744,300_14,I-country


In [ ]:
valid_9810

,id_word,slot
0,1_1,O
1,1_2,O
2,1_3,B-music_item
3,1_4,O
4,1_5,B-playlist_owner
...,...,...
2741,300_11,B-city
2742,300_12,I-country
2743,300_13,O
2744,300_14,B-country


In [ ]:
#0.98798, ,  
from sklearn.metrics import f1_score
f1_score(sample["slot"],valid_0977["slot"],average="micro")

0.9799708667152222

In [ ]:
sample.to_csv("bert_base_case_11.csv",index=False)